In [1]:
import cv2
from video import Video
import pprint



In [2]:
# Open the video file
path = "videos/tv9_video.mp4"
video = Video(path)

In [3]:

from textExtraction import frame_gap
from textExtraction import stitch_strings
from regionofinterest import scrolling_region

In [4]:
import cv2
import pytesseract
from PIL import Image

In [5]:
def attatch_strings(text1, text2):
    # case 0: text1 is empty, return text2
    if len(text1) == 0:
        return text2
    # case 1: last words in text1 and text2 are the same , return text2
    if text1[-1] == text2[-1]:
        return text1
    # case 2: last words in text1 is the penultimate word in text2, add the last wors to text2
    elif text1[-1] == text2[-2]:
        return text1 + text2[-1:]
    # case 3: search for the last word of text1 in text2 if found, add the words to the right of the penultimate word in text2 to text1 after removing the last word from text1
    elif text1[-2] in text2:
        # search the index of the last word of text1 in text2 if threre are multiple occurences, choose the last one
        index = -1
        for i, word in enumerate(text2):
            if word == text1[-2]:
                index = i
        return text1[:-1] + text2[index+1:]
    # case 4: serach for the last word in text1 in text2, if found, remove the last word in text1
    elif text1[-2] not in text2:
        return text1[:-1] + text2


In [6]:
def text_from_image(image, window):
    x, y, w, h = window

    # get roi from image
    roi = image[y : y + h, x : x + w]

    # do gray scale
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Perform thresholding
    _, thresholded_roi = cv2.threshold(
        gray_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # Convert the image to PIL format
    pil_image = Image.fromarray(thresholded_roi)

    # Set the language to Telugu
    custom_config = r"--oem 3 --psm 6 -l tel"
    # Perform OCR on the cropped image to get text and confidence
    telugu_text = pytesseract.image_to_data(pil_image, config=custom_config, output_type=pytesseract.Output.DICT)
    # strip the first and last few characters to avoid noise
    # strip_front, strip_back = stripping_lengths(telugu_text)

    # telugu_text = telugu_text[strip_front:-strip_back]


    return telugu_text

In [7]:
final_string = ""
x, y, w, h = scrolling_region

# Read frames from the video
for i in range(video._get_frame_count()):
    # Read the next frame
    frame = video.frame()

    if frame is None:
        break

    # Extract text from every 10th frame using extract_telugu_text
    if i % 25 == 0:
        telugu_text = text_from_image(frame, (x, y, w, h))
        telugu_text = telugu_text['text']
        # remove last word
        if len(telugu_text) > 0:
            telugu_text = telugu_text[:-1]
        # remove empty strings
        telugu_text = [line for line in telugu_text if line.strip()]

        

        print( telugu_text)

        # Append the extracted text to the text_list
        # print("current frame:", telugu_text)
        # text_list.append(telugu_text)
        final_string = attatch_strings(final_string, telugu_text)
        print("final string:", " ".join(final_string))

['ట్లో', 'రాత్రి', '9గంటలకు', '//', 'అన్ని', 'రకాల', 'వార్తల', 'కోసం', 'చూస్తూనే', 'ఉండండి']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే ఉండండి
['గంటలకు', '//', 'అన్ని', 'రకాల', 'వార్తల', 'కోసం', 'చూస్తూనే', 'ఉండండి']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే ఉండండి
['//', '_', 'అన్ని', 'రకాల', 'వార్తల', 'కోసం', 'చూస్తూనే', 'ఉండండి']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే ఉండండి
['న్ని', 'రకాల', 'వార్తల', 'కోసం', 'చూస్తూనే', 'ఉండండి']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే ఉండండి
['వార్తల', 'కోసం', 'చూస్తూనే', 'ఉండండి']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే ఉండండి
['కో', 'నెంబర్\u200c', 'వన్\u200c', 'న్యూస్\u200c', 'నెట్\u200cవర్క్\u200c:']
final string: ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే కో నెంబర్‌ వన్‌ న్యూస్‌ నెట్‌వర్క్‌:
['క్లో', 'నెంబర్\u200c', 'వన్\u200c', 'న్యూస్\u200c', 'నెట్\u200cవర్క్\u200c:', 'నెంబర్\u200c', 'వన్\u

In [8]:
print(final_string)

['ట్లో', 'రాత్రి', '9గంటలకు', '//', 'అన్ని', 'రకాల', 'వార్తల', 'కోసం', 'చూస్తూనే', 'కో', 'నెంబర్\u200c', 'వన్\u200c', 'న్యూస్\u200c', 'నెట్\u200cవర్క్\u200c:', 'నెంబర్\u200c', 'వన్\u200c', 'న్యూస్\u200c', 'ఛానల్\u200c', 'టీవీ?', 'తెలుగు', 'నిరంతరం', 'ప్రజల', 'కోసం,', 'మెరుగైన', 'సమాజం', 'కోసం', 'చూడండి', '౪9', 'తెలుగు.', '//', '666', '820%:', 'మీ', 'ప్రాంతంలో', 'సమస్యలేమైనా', 'ఉంటే', 'మా', '%411', '10.', '1౧(0్రు[/9.00౧', '10.', '1౧10(్రగ/.000', 'తగ్\u200c', 'మార్నింగ్\u200c', 'ప్రైమ్\u200c', 'టైమ్\u200c', '1]', 'గగ', 'మార్నింగ్\u200c', 'ప్రైమ్\u200c', 'టైమ్\u200c.', 'మార్షింగ్\u200c', 'ప్రైమ్\u200c', 'టైమ్\u200c', '.', 'ఉదయం', 'ర', 'నుంచి', '11గంటల', 'వరకు', 'బ్రేకింగ్\u200c', 'న్యూస్\u200c,', 'స్పీడ్\u200c', 'న్యూస్\u200c...', 'మీ', 'కోసం', '5గంటలపాటు', 'నాన్\u200cస్టాప్\u200cగా', 'మీ', 'ముందుకు', '1౪9', 'మార్నింగ్\u200c', 'ప్రైమ్\u200c', 'టైమ్\u200c.', '//', 'థొ', 'ర్నింగ్\u200c', 'ప్రైమ్\u200c', 'టైమ్\u200c.', '//', 'శ్ర', 'ఇన్\u200cసైడ్\u200c', 'రాజకీయాలపై', 'పొలిటికల్\u200c', 'మి

In [9]:
# from the words list create a string
final_string = " ".join(final_string)
print(final_string)

ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే కో నెంబర్‌ వన్‌ న్యూస్‌ నెట్‌వర్క్‌: నెంబర్‌ వన్‌ న్యూస్‌ ఛానల్‌ టీవీ? తెలుగు నిరంతరం ప్రజల కోసం, మెరుగైన సమాజం కోసం చూడండి ౪9 తెలుగు. // 666 820%: మీ ప్రాంతంలో సమస్యలేమైనా ఉంటే మా %411 10. 1౧(0్రు[/9.00౧ 10. 1౧10(్రగ/.000 తగ్‌ మార్నింగ్‌ ప్రైమ్‌ టైమ్‌ 1] గగ మార్నింగ్‌ ప్రైమ్‌ టైమ్‌. మార్షింగ్‌ ప్రైమ్‌ టైమ్‌ . ఉదయం ర నుంచి 11గంటల వరకు బ్రేకింగ్‌ న్యూస్‌, స్పీడ్‌ న్యూస్‌... మీ కోసం 5గంటలపాటు నాన్‌స్టాప్‌గా మీ ముందుకు 1౪9 మార్నింగ్‌ ప్రైమ్‌ టైమ్‌. // థొ ర్నింగ్‌ ప్రైమ్‌ టైమ్‌. // శ్ర ఇన్‌సైడ్‌ రాజకీయాలపై పొలిటికల్‌ మిర్చి సా:


In [10]:
print(len(final_string))

584


In [11]:
news_stories = []

# Split the text into news stories
for story in final_string.split("।"):
    news_stories.append(story)

In [13]:
print(news_stories)

['ట్లో రాత్రి 9గంటలకు // అన్ని రకాల వార్తల కోసం చూస్తూనే కో నెంబర్\u200c వన్\u200c న్యూస్\u200c నెట్\u200cవర్క్\u200c: నెంబర్\u200c వన్\u200c న్యూస్\u200c ఛానల్\u200c టీవీ? తెలుగు నిరంతరం ప్రజల కోసం, మెరుగైన సమాజం కోసం చూడండి ౪9 తెలుగు. // 666 820%: మీ ప్రాంతంలో సమస్యలేమైనా ఉంటే మా %411 10. 1౧(0్రు[/9.00౧ 10. 1౧10(్రగ/.000 తగ్\u200c మార్నింగ్\u200c ప్రైమ్\u200c టైమ్\u200c 1] గగ మార్నింగ్\u200c ప్రైమ్\u200c టైమ్\u200c. మార్షింగ్\u200c ప్రైమ్\u200c టైమ్\u200c . ఉదయం ర నుంచి 11గంటల వరకు బ్రేకింగ్\u200c న్యూస్\u200c, స్పీడ్\u200c న్యూస్\u200c... మీ కోసం 5గంటలపాటు నాన్\u200cస్టాప్\u200cగా మీ ముందుకు 1౪9 మార్నింగ్\u200c ప్రైమ్\u200c టైమ్\u200c. // థొ ర్నింగ్\u200c ప్రైమ్\u200c టైమ్\u200c. // శ్ర ఇన్\u200cసైడ్\u200c రాజకీయాలపై పొలిటికల్\u200c మిర్చి సా:']
